In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import pandas as pd

# Import the CRUD module
from crud import AnimalShelter

# Initialize the AnimalShelter object
shelter = AnimalShelter()

# Fetch data from the MongoDB collection
df = pd.DataFrame.from_records(shelter.read({}))  # Read all records from MongoDB

# Remove the '_id' column to avoid issues with ObjectID
df.drop(columns=['_id'], inplace=True)

# Ensure that 'age_upon_outcome_in_weeks' only contains positive values for marker sizes
df['age_upon_outcome_in_weeks'] = df['age_upon_outcome_in_weeks'].clip(lower=0)  # Replace negative values with 0

# Initialize the Dash app
app = JupyterDash('Animal Shelter Dashboard')

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1('Grazioso Salvare Dashboard'),  # Title of the dashboard
    html.Img(src='Grazioso Salvare Logo.png', style={'height': '100px', 'width': 'auto'}),  # Logo
    html.Div(children='Created by Cody VanGosen'),  # Unique identifier

    # Interactive filter options for rescue types
    dcc.Dropdown(
        id='rescue-type-dropdown',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset'  # Default value
    ),

    # Interactive Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",  # Enable filtering by columns
        sort_action="native",  # Enable sorting of columns
        page_size=10,  # Display 10 rows per page
        row_selectable="single",  # Enable single row selection
        style_table={'overflowX': 'auto'},  # Horizontal scrolling for wide tables
        style_cell={'textAlign': 'left'},  # Left align text in cells
        style_data_conditional=[  # Conditional formatting for better visual cues
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            },
            {
                'if': {'row_index': 'even'},
                'backgroundColor': 'rgb(255, 255, 255)'
            }
        ]
    ),

    html.Br(),
    html.Div(id='map-id', className='col s12 m6'),  # Div to hold the map output
])

# Callback to update the data table and charts based on selected rescue type
@app.callback(
    [Output('datatable-id', 'data'),
     Output('geo-chart', 'figure'),
     Output('second-chart', 'figure')],
    [Input('rescue-type-dropdown', 'value')]
)
def update_dashboard(selected_rescue_type):
    # Check if the 'rescue_type' column exists
    if 'rescue_type' not in df.columns:
        raise ValueError("The 'rescue_type' column is missing from the dataset.")

    # Filter data based on the selected rescue type
    if selected_rescue_type == 'Reset':
        filtered_data = df
    else:
        filtered_data = df[df['rescue_type'] == selected_rescue_type]

    # Update data table
    data = filtered_data.to_dict('records')

    # Create geolocation chart
    geo_fig = px.scatter_mapbox(filtered_data, lat='location_lat', lon='location_long', color='breed',
                                 size='age_upon_outcome_in_weeks', mapbox_style='carto-positron')

    # Create second chart (e.g., pie chart)
    second_fig = px.pie(filtered_data, names='breed', values='age_upon_outcome_in_weeks')

    return data, geo_fig, second_fig

# Callback to update the geolocation map based on the selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10)

    dff = pd.DataFrame(viewData)  # Convert the viewData (list of dictionaries) into a DataFrame
    row = index[0] if index else 0  # Get the selected row; if no row is selected, default to the first row

    # Retrieve latitude and longitude from the selected row
    latitude = dff.iloc[row, 13]
    longitude = dff.iloc[row, 14]

    # Return a map centered at the selected location with a marker
    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[latitude, longitude], children=[
            dl.Tooltip(dff.iloc[row, 4]),  # Tooltip shows the animal's breed
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row, 9])  # Popup shows the animal's name
            ])
        ])
    ])

# Run the Dash app in the Jupyter notebook
app.run_server(mode='inline', debug=True, port=8056)  # Use a different port if necessary


MongoDB connection established successfully.
Found 7454 document(s) matching the query.


---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File /usr/local/anaconda/lib/python3.9/site-packages/pandas/core/indexing.py:960, in _LocationIndexer.__getitem__(
    self=<pandas.core.indexing._iLocIndexer object>,
    key=(0, 13)
)
    958     key = tuple(com.apply_if_callable(x, self.obj) for x in key)
    959     if self._is_scalar_access(key):
--> 960         return self.obj._get_value(*key, takeable=self._takeable)
        key = (0, 13)
        self = <pandas.core.indexing._iLocIndexer object at 0x7f3fdb718180>
        self._takeable = True
    961     return self._getitem_tuple(key)
    962 else:
    963     # we by definition only have the 0th axis

File /usr/local/anaconda/lib/python3.9/site-packages/pandas/core/frame.py:3612, in DataFrame._get_value(
    self=Empty DataFrame
Columns: []
Index: [],
    index=0,
    col=13,
    takeable=True
)
   3593 """
   3594 Quickly retr